<style>
    .center-text{
        text-align: center;
    }
</style>

<div class='center-text'>

# Tarea 1 Aprendizaje Automático

### Julian Ricardo Salazar duarte

#### Universidad Nacional de Colombia

#### 2024-1
</div>


- [1. Preprocesamiento de datos](#s1)
- [2. Análisis exploratorio de datos](#s2)


In [124]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

<a id="s1"></a>
### 1. Preprocesamiento de datos

In [125]:
def load_data(filename, skiprows=0, sheet_name=None):
    if sheet_name is None:
        data = pd.read_excel(filename, skiprows=skiprows)
    else:
        data = pd.read_excel(filename, skiprows=skiprows, sheet_name=sheet_name)
    return data

In [126]:
def clean_df(df):

    df = df.dropna(how='all')

    for i in range (len(df.columns)):
        if 'Unnamed' in df.columns[i]:
            df = df.rename(columns={df.columns[i]: df.columns[i-1].split()[0] + ' ' + df.iloc[0, i]})
        elif not pd.isna(df.iloc[0, i]):
            df = df.rename(columns={df.columns[i]: df.columns[i] + ' ' + df.iloc[0, i]})

    df = df.drop(df.index[0])
    df = df.dropna(how='any')
    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.capitalize()
    df['Fecha'] = df['Fecha'].apply(str)

    df = df.rename(columns={df.columns[2]: 'Marca'})
    
    return df

In [127]:
df = load_data('CC FT 17 Formato de Control de Calidad Café de Trillado.xlsx',
               5, ['CONTROL CALIDAD CAFE TRILLADO J', 'Sheet2'])

df1 = df['CONTROL CALIDAD CAFE TRILLADO J']
df2 = df['Sheet2']

df1 = clean_df(df1)
df2 = clean_df(df2)

In [128]:
df_trillado = pd.concat([df1, df2], axis=0)

for col in df_trillado.columns:
    if isinstance(df_trillado[col].iloc[0], str):
        df_trillado[col] = df_trillado[col].str.lower().str.strip()
    if len(set(df_trillado[col])) == 1:
        df_trillado = df_trillado.drop(columns=[col])

df_trillado['Lote'] = df_trillado['Lote'].str.replace('-', '', regex=False).str.lstrip('0')

df_trillado = df_trillado.reset_index(drop=True)

print(df_trillado.shape)

(106, 9)


##### Label (X) e input (Y)


In [129]:
def replace_date(date):
    date = date.split('-')
    if date[1].isdigit():
        if int (date[1]) < 4:
            return 1
        elif int (date[1]) < 7:
            return 2
        elif int (date[1]) < 10:
            return 3
        else:
            return 4
    else:
        if 'ene' in date[1].lower() or 'feb' in date[1].lower() or 'mar' in date[1].lower():
            return 1
        elif 'abr' in date[1].lower() or 'may' in date[1].lower() or 'jun' in date[1].lower():
            return 2
        elif 'jul' in date[1].lower() or 'ago' in date[1].lower() or 'sep' in date[1].lower():
            return 3
        else:
            return 4

In [130]:
# Fecha toma los valores entre 1 y 4 dividiendo el año en trimestres

df_trillado['Fecha'] = df_trillado['Fecha'].apply(replace_date)

In [131]:
def replace_lote(lote):
    if len(lote) == 7:
        return int(lote[0])
    else:
        return int(lote[:2])

In [132]:
# La columna Lote tiene valores enteros en entre 1 y 23 representando la marca o tipo de cafe

df_trillado['Lote'] = df_trillado['Lote'].apply(replace_lote)

In [133]:
tmp = df_trillado[['Lote', 'Marca']]
tmp = tmp.drop_duplicates(subset='Lote')
print(tmp)

    Lote                       Marca
0      1                 madre laura
1      9                tabi natural
2     10                   don mario
3      7                   don felix
7      8         gesha villabernarda
8      2         madre laura natural
9      5                  doña dolly
12     3    madre laura descafeinado
17    11      monteverde - wush wush
18    12     el ocaso - pink bourbon
28    13   el cedrela - pink bourbon
38    16       gesha villa - natural
39    15         don victor - marago
42     6                  don rafael
43     4                doña rosalba
49    17      familia bedoya castaño
54    18                   don johan
57    19     gesha blue - monteverde
58    20  bourbon sidra - las flores
59    21    don victor - red bourbon
70    22         el ocaso - caturron
72    23             esteban robledo


In [134]:
df = load_data('CC FT 17 Formato de Control de Calidad Café de Trillado.xlsx', sheet_name=['LOTES'])
df = df['LOTES']
df = df[['N°', 'TIPO DE CAFÉ']]
df['TIPO DE CAFÉ'] = df['TIPO DE CAFÉ'].str.lower()
df = df.dropna(how='any')
df = df.set_index('N°')
dict_lote = df.to_dict()['TIPO DE CAFÉ']
dict_lote[23] = tmp[tmp['Lote'] == 23]['Marca'].values[0]

In [135]:
for key, value in dict_lote.items():
    print(f'{key}: {value}')

1: madre laura lavado
2: madre laura natural
3: madre laura descafeinado
4: doña rosalba 
5: doña dolly
6: don rafael pbw 
7: don felix
8: gesha villabernarda
9: don juan tabi
10: doña liceth 
11: monteverde - wush wush
12: el ocaso - pink bourbon
13: el cedrela - pink bourbon 
14: don reinaldo 
15: don victor - maragogipe 
16: gesha natural villabernarda
17: familia bedoya castaño
18: don johan 
19: familia gutierrez - gesha 
20: familia vergara - bourbon sidra 
21: don victor - red bourbon
22: el ocaso - caturron natural 
23: esteban robledo


In [136]:
# se elimina la columna marca ya que es redundante con la columna Lote

df_trillado = df_trillado.drop(columns=['Marca'])

In [137]:
# normalizar la columna Cantidad
scaler = MinMaxScaler((0.5, 8))
df_trillado['Cantidad'] = scaler.fit_transform(df_trillado[['Cantidad']])

In [138]:
# cantidad, %h y mallas # se deja igual (se normaliozara más adelante)

tmp = df_trillado['Notas de catación']
tmp = tmp.str.split(',', expand=True)

In [139]:
def separe_y(row):
    new_row = []
    for r in row:
        tmp = r.split(' y ')
        new_row.extend(tmp)
    return new_row


In [140]:
def delete_and_normalize(row):
    
    new_row = []
    for r in row:
        if (r == 'caramleo' or r == 'balanaceado' or 'fnal' in r or 'final' in r or 
            'residual' in r or 'cuepo' in r or 'prolongado' in r):
            continue
        elif 'acidez' in r or 'cuerpo' in r:
            pos = r.find(' ', 7)
            if pos == -1:
                new_row.append(r.replace('.', ''))
            else:
                new_row.append(r[:pos])
        elif 'mora' in r:
            new_row.append('mora')
        elif 'miel' in r:
            new_row.append('miel')
        elif 'fresa' in r:
            new_row.append('fresa')
        elif 'chocolate' in r:
            new_row.append('chocolate')
        elif 'cacao' in r:
            new_row.append('cacao')
        elif 'panela' in r:
            new_row.append('panela')
        else:
            new_row.append(r.replace('.', ''))
    return new_row

In [141]:
# Notas de catacion se aplicará One-Hot Encoding

notes = set()
for i in range(tmp.shape[1]):
    notes = notes.union(set(tmp[i].str.strip()))

notes = list(notes)
notes = [note for note in notes if note is not None]
notes = separe_y(notes)
notes = delete_and_normalize(notes)
notes = list(set(notes))
notes.sort()

In [142]:
notes

['acidez brillante',
 'acidez citrica',
 'acidez jugosa',
 'acidez malica',
 'acidez media',
 'almendras',
 'arandanos',
 'azucar morena',
 'cacao',
 'caramelo',
 'caña de azucar',
 'cedro',
 'chocolate',
 'ciruela madura',
 'clavos de olor',
 'cuerpo aterciopelado',
 'cuerpo cremoso',
 'cuerpo medio',
 'cuerpo sedoso',
 'durazno',
 'flor de naranja',
 'frambuesa',
 'fresa',
 'frutas amarillas',
 'frutas maduras',
 'frutos rojos',
 'jalea',
 'jazmin',
 'lavanda',
 'limon',
 'limonaria',
 'limonciollo',
 'mandarina',
 'manzana roja',
 'maracuya',
 'menta',
 'miel',
 'mora',
 'naranja',
 'nueces',
 'nuez moscada',
 'panela',
 'rosas',
 'sandia',
 'te de cedron',
 'toque frutal',
 'uva',
 'vainilla']

In [143]:
for col in notes:
    df_trillado[col] = np.nan

In [144]:
for i in notes:
    for j in df_trillado.index:
        if i in df_trillado.at[j, 'Notas de catación']:
            df_trillado.at[j, i] = 1
        else:
            df_trillado.at[j, i] = 0

In [145]:
# Se eliminan las columnas Notas de catación ya que se aplicó One-Hot Encoding

df_trillado = df_trillado.drop(columns=['Notas de catación'])

In [146]:
# responsable sa cambia lfq por 1 y ac por 0

for i in df_trillado.index:
    if df_trillado.at[i, 'Responsable'] == 'lfq':
        df_trillado.at[i, 'Responsable'] = 1
    else:
        df_trillado.at[i, 'Responsable'] = 0

In [147]:
df_trillado.head()

,Fecha,Lote,Cantidad,%h %,Mallas #,Puntaje n°,Responsable,acidez brillante,acidez citrica,acidez jugosa,...,naranja,nueces,nuez moscada,panela,rosas,sandia,te de cedron,toque frutal,uva,vainilla
0,3,1,8.000000,10.9,14,84,1,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,3,9,2.496763,10.2,14,85,1,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,10,2.114185,10.7,14,84.5,1,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,7,0.500000,10.5,14,84.5,1,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,1,1.525603,10.7,14,84,1,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [148]:
# dividir datos en features y target

Y = df_trillado['Puntaje n°']

X = df_trillado.drop(columns=['Puntaje n°'])

Se buscara predecir el puntaje de los granos  de café (y) en base de las caracteristicas Fecha, Lote, cantidad, %h, Mallas y notas de catación (x). Para ello se realizó un preprocesamiento de los datos, con el fin de tener un dataset limpio y listo para ser utilizado en modelos de ml.

In [149]:
for col in df_trillado.columns:
    print(f'{col}: {len(set(df_trillado[col]))}')


Fecha: 4
Lote: 22
Cantidad: 66
%h %: 13
Mallas #: 3
Puntaje n°: 10
Responsable: 2
acidez brillante: 2
acidez citrica: 2
acidez jugosa: 2
acidez malica: 2
acidez media: 2
almendras: 2
arandanos: 2
azucar morena: 2
cacao: 2
caramelo: 2
caña de azucar: 2
cedro: 2
chocolate: 2
ciruela madura: 2
clavos de olor: 2
cuerpo aterciopelado: 2
cuerpo cremoso: 2
cuerpo medio: 2
cuerpo sedoso: 2
durazno: 2
flor de naranja: 2
frambuesa: 2
fresa: 2
frutas amarillas: 2
frutas maduras: 2
frutos rojos: 2
jalea: 2
jazmin: 2
lavanda: 2
limon: 2
limonaria: 2
limonciollo: 2
mandarina: 2
manzana roja: 2
maracuya: 2
menta: 2
miel: 2
mora: 2
naranja: 2
nueces: 2
nuez moscada: 2
panela: 2
rosas: 2
sandia: 2
te de cedron: 2
toque frutal: 2
uva: 2
vainilla: 2


<a id="s2"></a>
### 2. Análisis exploratorio de datos